In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df=pd.read_csv('synthetic_study_drift_dataset.csv')

In [3]:
df.head()

,date,study_hours,break_minutes,focus_score,difficulty_level,time_of_day,topic_consistency,quiz_score
0,2025-01-01,4.596057,56.865479,4,1,Morning,1,82.293737
1,2025-01-02,3.834083,31.359188,2,1,Afternoon,1,75.034495
2,2025-01-03,4.777226,66.041915,2,2,Morning,0,69.281940
3,2025-01-04,5.827636,23.972234,5,2,Night,0,87.740956
4,2025-01-05,3.719016,53.802856,2,1,Afternoon,0,67.214794
